# Widgets in Kernels
In addition to using kernels in code, you can interact with the widgets created by a notebook, or even multiple notebooks.

> This approach probably won't work for complex widgets like bqplot (yet), but likely the implementation can improve.

In [1]:
import ktop

In [2]:
nb = ktop.Notebook()
nb.nbformat.cells = list(map(ktop.Code, [
    """from ipywidgets import FloatSlider, VBox\nfrom traitlets import dlink\nimport math""",
    """x, y, z = [FloatSlider(value=1.0, description='${}$'.format(a), min=-1.0, max=1.0) for a in "xyz"]""",
    """dlink((x, "value"), (y, "value"), math.sin)\ndlink((x, "value"), (z, "value"), math.cos)""",
]))
yak = nb.run_one(shutdown=False, reuse=False)
zebra = nb.run_one(shutdown=False, reuse=False)

In [3]:
yak_widgets = [w for w in yak.widgets if hasattr(w, "layout")]
zebra_widgets = [w for w in zebra.widgets if hasattr(w, "layout")]
ktop.W.HBox([ktop.W.VBox(yak_widgets), ktop.W.VBox(zebra_widgets)])

Because these widgets live in both the host kernel (that imported `ktop`) and the guest kernel, these can be `link`ed.

In [4]:
xyz = ktop.W.FloatSlider(description="$\Sigma _{zebra}  \Sigma _{yak}$", min=-9, max=9)
def _update_xyz(_):
    xyz.value = sum([w.value for w in yak_widgets]) * sum([w.value for w in zebra_widgets])
[w.observe(_update_xyz) for w in yak_widgets + zebra_widgets]
xyz

FloatSlider(value=0.0, description='$\\Sigma _{zebra}  \\Sigma _{yak}$', max=9.0, min=-9.0)

While these examples are all based on `ipykernel`, theoretically any kernel implementing the widget message spec should work.